# LLM as the entrypoint

- Inspired from https://blog.apiad.net/p/building-a-perplexity-ai-clone and others to build Perplexity.ai clones

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages.ai import AIMessage

from pydantic import BaseModel
from typing import List, Field

class QueryExpansion(BaseModel):
    interpretation: str = Field(description="interpretation of the user's information need(s)")
    queries: List[str] = Field(description="list of relevant Google searches")

_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
Given the following question, provide a set of {num_queries} relevant Google searches that would answer the question. 
First think about the user's question and provide your own interpretation of the information need(s).
Then generate the relevant queries.
            """,
        ),
        (
            "human",
            """
User query: {query}
            """,
        ),
    ]
)



llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
runnable = _prompt | llm.with_structured_output(QueryExpansion)
result = runnable.invoke(
    {
        "query": "Is 98point6 a good company to work for?",
        "num_queries": 5
    }
)

print(result)